The program is not ready yet!

In [1]:
!pip install mmcv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.2/473.2 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.1/372.1 kB 26.5 MB/s eta 0:00:00
  Created wheel for mmcv: filename=mmcv-2.0.0-cp310-cp310-linux_x86_64.whl size=1359221 sha256=6f4c4a368c45bab6caeffbfb0c65ab2cc783952f593d2fe91f86adb82e4a7b5a
  Stored in directory: /root/.cache/pip/wheels/19/1b/83/aca08aea56449d2284a1025b6e7a762dec954827816a38109d
Successfully built mmcv


In [2]:
!pip install mmocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 618.1/618.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 24.2 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ | / done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
  Created wheel for pycocotools: filename=pycocotools-2.0.6-cp310-cp310-linux_x86_64.whl size=102203 sha256=28fb6a2d992a7ad67ad32bff160308987226839154854d20b5ae715754260e58
  Stored in directory: /root/.cache/pip/wheels/58/e6/f9/f87c8f8be098b51b616871315318329cae12cdb618f4caac93
Successfully built pycocotools


In [3]:
!pip install mmdet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.7 MB/s eta 0:00:00


In [4]:
#!pip install joblib

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
#from sklearn.metrics import f1_score, roc_auc_score
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split, DataLoader, Dataset
import torchvision.transforms as transforms
from torchvision.io import read_image, ImageReadMode
import torchvision
from torchvision.datasets.vision import VisionDataset
from tqdm.notebook import tqdm
from PIL import Image
from glob import glob
import os
import json
import joblib
import mmocr
from mmocr.models import textrecog
#help(textrecog)

In [6]:
#with open('mmocr.joblib','wb') as f:
#    joblib.dump(mmocr,f)
#f.close()

In [7]:
def image_convrt(img):
    img = img.clone().cpu().numpy()
    img = img.transpose(1,2,0)
    std = [0.5,0.5,0.5]
    mean = [0.5,0.5,0.5]
    img = img*std + mean
    return img

In [8]:
#Функция для воссоздания маски изображения из файла json
def interesting(path,dimensions):
    def typization(string):
        if string=='chart_title':
            return (1,1,1)
        elif string=='axis_title':
            return (2,2,2)
        elif string=='tick_label':
            return (3,3,3)
        elif string=='plot-bb':
            return (4,4,4)
        elif string=='axes':
            return (5,5,5)
        elif string=="bars":
            return (6,6,6)
        elif string=="boxplots":
            return (7,7,7)
        elif string=="dot points":
            return (8,8,8)
        elif string=="lines":
            return (9,9,9)
        elif string=="scatter points":
            return (10,10,10)
#    creator = np.array(dimensions[1]*[dimensions[2]*[0]])
    creator = np.zeros((dimensions[1], dimensions[2]), dtype=np.uint8)
    creator = cv2.cvtColor(creator, cv2.COLOR_GRAY2BGR)
    print('img: ', dimensions,'\n')
    s=""
    with open(path) as f:
        s=f.read()
    gen=json.loads(s)
    gen2=gen['plot-bb']
    cv2.fillPoly(creator, [np.array([[(gen2['x0']+gen2['width'],gen2['y0'])],[(gen2['x0'],gen2['y0'])],[(gen2['x0'],gen2['y0']+gen2['height'])],[(gen2['x0']+gen2['width'],gen2['y0']+gen2['height'])]],dtype=np.int32)], typization('plot-bb'))
    for i in range(len(gen['text'])):
        if len(gen['text'][i]['polygon'])!=0:
            gen1=gen['text'][i]['polygon']
            cv2.fillPoly(creator, [np.array([[(gen1['x0'],gen1['y0'])],[(gen1['x1'],gen1['y1'])],[(gen1['x2'],gen1['y2'])],[(gen1['x3'],gen1['y3'])]],dtype=np.int32)], typization(gen['text'][i]['role']))
    axe=gen['axes']
    axex=axe['x-axis']['ticks']
    axey=axe['y-axis']['ticks']
    for i in range(len(axex)):
        cv2.fillPoly(creator, [np.array([[(axex[i]['tick_pt']['x'],axex[i]['tick_pt']['y'])],[(axex[i]['tick_pt']['x'],axex[i]['tick_pt']['y']-gen2['height'])]],dtype=np.int32)], typization('axes'))
    for i in range(len(axey)):
        cv2.fillPoly(creator, [np.array([[(axey[i]['tick_pt']['x'],axey[i]['tick_pt']['y'])],[(axey[i]['tick_pt']['x']+gen2['width'],axey[i]['tick_pt']['y'])]],dtype=np.int32)], typization('axes'))
    if len(gen["visual-elements"]["bars"])!=0:
        for el in gen["visual-elements"]["bars"]:
            cv2.fillPoly(creator, [np.array([[(el['x0']+el['width'],el['y0'])],[(el['x0'],el['y0'])],[(el['x0'],el['y0']+el['height'])],[(el['x0']+el['width'],el['y0']+el['height'])]],dtype=np.int32)], typization('bars'))
    elif len(gen["visual-elements"]["boxplots"])!=0:
        ...
    elif len(gen['visual-elements']['dot points'])!=0:
        for i in gen['visual-elements']['dot points'][0]:
            cv2.circle(creator, (int(i['x']), int(i['y'])), 4, typization('dot points'), 8)
    elif len(gen['visual-elements']['lines'])!=0:
        k=[]
        for i in gen['visual-elements']['lines'][0]:
            cv2.circle(creator, (int(i['x']), int(i['y'])), 2, typization('dot points'), 5)
        for i in range(1,len(gen['visual-elements']['lines'][0])):
            cv2.drawContours(creator,[np.array([(int(gen['visual-elements']['lines'][0][i-1]['x']),int(gen['visual-elements']['lines'][0][i-1]['y'])),(int(gen['visual-elements']['lines'][0][i]['x']),int(gen['visual-elements']['lines'][0][i]['y']))])],-1,typization('lines'),3)
    elif len(gen['visual-elements']['scatter points'])!=0:
        for i in gen['visual-elements']['scatter points'][0]:
            cv2.circle(creator, (int(i['x']), int(i['y'])), 2, typization('dot points'), 5)
#    cv2.imwrite("mask.jpg",creator)
#    polygon_detection.script(img_exampl,creator)
    creator = cv2.cvtColor(creator, cv2.COLOR_BGR2GRAY)
    creator = np.array([creator]*3)
    output = torch.from_numpy(creator)
    return output
#

In [9]:
def reconstruct(img: np.ndarray, mask: np.ndarray, tup: tuple):
    if tup == (128, 64, 128): #Chart title
        coords = np.where(mask[:,:,] == tup) #Ищем координаты полигона

        y_begin, y_end = coords[0].min(), coords[0].max() #Выделяем полигон
        x_begin, x_end = coords[1].min(), coords[1].max()

        return img[y_begin:y_end, x_begin:x_end] #Возвращаем обрезанное изображение

    if tup == (130, 76, 0): #Axis title
        coords = np.where(mask[:,:,] == tup) #Ищем координаты полигона
        fst_border = [] #Границы y координат
        snd_border = [] #Границы х координат

        for i in range(len(coords[0])):
            if coords[0][i] - coords[0][i + 1] < -1: #Все координаты идут в порядке возрастания, потому мы ищем резкий скачок значения в списке
                fst_border.append((coords[0][0], coords[0][i]))
                snd_border.append((coords[0][i + 1], coords[0].max()))
                break

        for i in range(len(coords[1])):
            if coords[1][i] - coords[1][i + 1] < -1:
                fst_border.append((coords[1][0], coords[1][i]))
                snd_border.append((coords[1][i + 1], coords[1].max()))
                break

        img1 = img[fst_border[0][0]:fst_border[0][1],fst_border[1][0]:fst_border[1][1]]
        img2 = img[snd_border[0][0]:snd_border[0][1],snd_border[1][0]:snd_border[1][1]]

        return img1, img2

    if tup == (0, 102, 0):
        coords = np.where(mask[:,:,] == tup) #Ищем координаты полигона
        y_tick_borders = []

        end_for_y_ticks = 0
        for i in range(len(coords[1])):
            if coords[1][i] - coords[1][i + 1] > 0:
                end_for_y_ticks = coords[1][i]
                break

        upper_border_y = coords[0][0]
        for i in range(len(coords[0])):
            try:
                if coords[0][i] - coords[0][i + 1] < -1:
                    y_tick_borders.append([(upper_border_y, coords[0][i]), (coords[1][0], end_for_y_ticks)])
                    upper_border_y = coords[0][i + 1]
            except BaseException:
                pass

        x_tick_border = img[upper_border_y:coords[0].max(), end_for_y_ticks:coords[1].max(),]

        return y_tick_borders, x_tick_border

    if tup == (112, 103, 87) or tup == (5, 5, 5):
        coords = np.where(mask[:,:,] == tup)

        y_begin, y_end = coords[0].min(), coords[0].max()
        x_begin, x_end = coords[1].min(), coords[1].max()

        return img[y_begin:y_end, x_begin:x_end + 1]

    if tup == (48, 41, 30):
        coords = np.where(mask[:,:,] == tup)
        bar_lower_border = np.where(mask[:,:,] == (0, 102, 0))[0].max()
        bar_upper_border = coords[0].min()
        bar_borders = []

        all_bar_polygons = np.unique(coords[1])

        left_border = all_bar_polygons[0]
        for i in range(len(all_bar_polygons)):
            try:
                if all_bar_polygons[i] - all_bar_polygons[i + 1] < -1:
                    bar_borders.append([(bar_upper_border, bar_lower_border), (left_border, all_bar_polygons[i])])
                    left_border = all_bar_polygons[i + 1]
            except BaseException:
                pass

        return bar_borders

    coords = np.where(mask[:,:,] == tup)

    y_begin, y_end = coords[0].min(), coords[0].max()
    x_begin, x_end = coords[1].min(), coords[1].max()

    return img[y_begin:y_end, x_begin:x_end + 1]

def script(img: np.ndarray, mask: np.ndarray):
    typ = {'chart_title': (128, 64, 128),
    'axis_title': (130, 76, 0),
    'tick_label': (0, 102, 0),
    'plot-bb': (112, 103, 87),
    'axes': (28, 42, 168),
    "bars": (48, 41, 30),
    "boxplots": (0, 50, 89),
    "dot points": (107, 142, 35),
    "lines": (70, 70, 70),
    "scatter points": (102, 102, 156)} #Цвет каждого полигона

    #Основной скрипт для восстановки полигонов
    ct = reconstruct(img, mask, typ["chart_title"])

    at1, at2 = reconstruct(img, mask, typ["axis_title"])
#    cv2.imwrite('./results/axis_title_y.jpg', at1)
#    cv2.imwrite('./results/axis_title_x.jpg', at2)

    tl_y, tl_x = reconstruct(img, mask, typ['tick_label'])
    y_tick_fragments = []
    for i in range(len(tl_y)):
        up, down = tl_y[i][0][0], tl_y[i][0][1]
        left, right = tl_y[i][1][0], tl_y[i][1][1]

        fragment = img[up:down, left:right,]

        y_tick_fragments.append(fragment)
    x_tick_fragments = tl_x

    pb = reconstruct(img, mask, typ["plot-bb"])

    axes = reconstruct(img, mask, typ["axes"])

    try:
        bars = reconstruct(img, mask, typ['bars'])
        bar_fragments = []
        for i in range(len(bars)):
            up, down = bars[i][0][0], bars[i][0][1]
            left, right = bars[i][1][0] - 30, bars[i][1][1]

            fragment = img[up:down,left:right,]
            bar_fragments.append(fragment)
        
        return ct, (at1, at2), (y_tick_fragments, x_tick_fragments), pb, axes, bar_fragments, 'bars'
    except BaseException:
        pass
    try:
        bp = reconstruct(img, mask, typ["boxplots"])
        boxplot_fragments = []
        for i in range(len(bp)):
            up, down = bars[i][0][0], bars[i][0][1]
            left, right = bars[i][1][0] - 30, bars[i][1][1]

            fragment = img[up:down,left:right,]
            boxplot_fragments.append(fragment)
        
        return ct, (at1, at2), (y_tick_fragments, x_tick_fragments), pb, axes, boxplot_fragments, 'boxplots'
    except BaseException:
        pass
    try:
        dp = reconstruct(img, mask, typ["dot points"])
        dp_fragments = []
        for i in range(len(dp)):
            up, down = bars[i][0][0], bars[i][0][1]
            left, right = bars[i][1][0] - 30, bars[i][1][1]

            fragment = img[up:down,left:right,]
            
            dp_fragments.append(fragment)
        return ct, (at1, at2), (y_tick_fragments, x_tick_fragments), pb, axes, dp_fragments, 'dot point'
    except BaseException:
        pass
    try:
        lines = reconstruct(img, mask, typ["lines"])
        line_fragments = []
        for i in range(len(lines)):
            up, down = bars[i][0][0], bars[i][0][1]
            left, right = bars[i][1][0] - 30, bars[i][1][1]

            fragment = img[up:down,left:right,]

            line_fragments.append(fragment)
        return ct, (at1, at2), (y_tick_fragments, x_tick_fragments), pb, axes, line_fragments, 'lines'
    except BaseException:
        pass
    try:
        sp = reconstruct(mask, typ['scatter points'])
        scatter_fragments = []
        for i in range(len(sp)):
            up, down = bars[i][0][0], bars[i][0][1]
            left, right = bars[i][1][0] - 30, bars[i][1][1]

            fragment = img[up:down,left:right,]
            scatter_fragments.append(fragment)
        return ct, (at1, at2), (y_tick_fragments, x_tick_fragments), pb, axes, scatter_fragments, 'scatter points'
    except BaseException:
        pass

In [10]:
image_path = '/kaggle/input/benetech-making-graphs-accessible/train/images'
mask_path = '/kaggle/input/benetech-making-graphs-accessible/train/annotations'
test_path = '/kaggle/input/benetech-making-graphs-accessible/train/images'
labels = pd.read_csv('/kaggle/input/benetech-making-graphs-accessible/sample_submission.csv')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(labels.head())
classes = ['bars','boxplots',"dot points","lines","scatter points"]#labels.name.values.tolist()
print(classes)
length = len(os.listdir(image_path))
#Тут куча закомментированного, но не удалённого (рудиментарного) кода
'''class Graphics(Dataset):
    def __init__(self, imgs_dir, masks_dir, count, is_val = False):
        self.imgs_dir = imgs_dir
        self.masks_dir = masks_dir

        imgs_paths = os.listdir(self.imgs_dir)
        imgs_paths.sort()

        mask_paths = os.listdir(self.masks_dir)
        mask_paths.sort()

        self.is_val = is_val
        if not is_val:  # для разделения на train/val в процессе
            self.imgs_paths = imgs_paths[:count]
            self.mask_paths = mask_paths[:count]
        else:
            self.imgs_paths = imgs_paths[-count:]
            self.mask_paths = mask_paths[-count:]

    def __len__(self):
        return len(self.imgs_paths)

    def __getitem__(self, idx):
        img = read_image(os.path.join(self.imgs_dir, self.imgs_paths[idx]), ImageReadMode.RGB)
        mask = interesting(os.path.join(self.masks_dir, self.mask_paths[idx]),img.shape)
#        mask = read_image(os.path.join(self.masks_dir, self.mask_paths[idx]), ImageReadMode.GRAY)

        return img, mask'''
    
torchvision.models.segmentation.DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1.transforms()
#Функция, предназначенная единственно чтобы изменять размер изображения (само изображение не меняется, т. к. важна точность до пикселя)
#def img_transform(img, mask, is_val=False, size=520):
#    img = img.float() / 255.0

#    one_dim = img.shape[2]
#    two_dim = img.shape[3]
#    one_dim1 = mask.shape[2]
#    two_dim1 = mask.shape[3]

#    dim1 = torch.Tensor([[(4096-one_dim)*[[0]]]]*3)
#    dim2 = torch.Tensor([[[(4096-two_dim)*[0]]]]*3)
#    dim3 = torch.Tensor([[(4096-one_dim1)*[[0]]]]*3)
#    dim4 = torch.Tensor([[[(4096-two_dim1)*[0]]]]*3)
#    dim1 = torch.Tensor([[(4096-one_dim)*[0]]]).unsqueeze(0).unsqueeze(0)
#    dim2 = torch.Tensor([[[(4096-two_dim)*[0]]]]).unsqueeze(0).unsqueeze(0)
#    dim3 = torch.Tensor([[(4096-one_dim1)*[0]]]).unsqueeze(0).unsqueeze(0)
#    dim4 = torch.Tensor([[[(4096-two_dim1)*[0]]]]).unsqueeze(0).unsqueeze(0)

#    img = torch.cat((torch.cat((img,dim1),dim=2),dim2),dim=3)
#    mask = torch.cat((torch.cat((mask,dim3),dim=2),dim4),dim=3)
#    img = img.to(device)

#    mask = mask.to(device)

#    return img, mask.squeeze(1).long()
def img_transform(img, mask, is_val=False, size=520):
    img = img.float() / 255.0
    mask = mask.float() / 255.0

    one_dim = img.shape[2]
    two_dim = img.shape[3]
    one_dim1 = mask.shape[2]
    two_dim1 = mask.shape[3]

    dim1 = torch.zeros((img.shape[0], img.shape[1], 1024 - one_dim, img.shape[3]))
    dim2 = torch.zeros((img.shape[0], img.shape[1], 1024, 1024 - two_dim))
    dim3 = torch.zeros((mask.shape[0], mask.shape[1], 1024 - one_dim1, mask.shape[3]))
    dim4 = torch.zeros((mask.shape[0], mask.shape[1], 1024, 1024 - two_dim1))

    img = torch.cat((torch.cat((img,dim1),dim=2),dim2),dim=3)
    mask = torch.cat((torch.cat((mask,dim3),dim=2),dim4),dim=3) # fixed dimension here
    img = img.to(device)

    mask = mask.to(device)

    return img, mask.squeeze(2).long()

img = torch.randn(2, 3, 100, 50)
mask = torch.randn(2, 3, 80, 70)

img_resized, mask_resized = img_transform(img, mask)

train_dataset_len = int(length * 0.7)
val_dataset_len = length - train_dataset_len

#train_dataset = Graphics(image_path, mask_path, train_dataset_len)

#val_dataset = Graphics(image_path, mask_path, val_dataset_len, is_val=True)

#print(train_dataset[5][0].shape)
#print(train_dataset[5][1].shape)
#img, mask = next(iter(train_dataset))

batch_size = 2
#Класс датасета, ещё не реализовал всё, что хотел
class ExperimentalDataset:
    def __init__(self,img_dir,mask_dir,batch,train='train'):
        self.newlist = []
        self.batch_size=batch
        k=0
#        print(os.listdir(img_dir))
#        idl=list(os.listdir(img_dir).sort())
#        mdl=list(os.listdir(mask_dir).sort())
#        for i in range(10):
#            print(idl[i],mdl[i])
#        ls = list(zip(idl,mdl))
        if train=='train':
#            self.newlist=ls[:int(0.9*len(ls))]
            for i in range(int(len(os.listdir(img_dir))*0.9)):   #os.listdir(img_dir)[:int(len(os.listdir(img_dir))*0.9)]
                namef = os.listdir(img_dir)[i]
                self.newlist.append((img_dir+'/'+namef,mask_dir+'/'+namef[:-4]+'.json'))
        elif train=='validation':
#            self.newlist=ls[:int(0.9*len(ls))]
            for i in range(int(len(os.listdir(img_dir))*0.1)):   #os.listdir(img_dir)[int(len(os.listdir(img_dir))*0.1):]
                namef = os.listdir(img_dir)[int(len(os.listdir(img_dir))*0.9)+i]
                self.newlist.append((img_dir+'/'+namef,mask_dir+'/'+namef[:-4]+'.json'))
        else:
#            self.newlist=ls
            for i in range((len(os.listdir(img_dir)))):
                self.newlist.append((os.listdir(img_dir)[i],os.listdir(mask_dir)[i]))
    
    def __len__(self):
        return len(self.newlist)
    
    def __getitem__(self,index):
        img=[]
        mask=[]
        transforming = transforms.ColorJitter(brightness=0.4,contrast=0.4)
        img.append(read_image(self.newlist[index+1][0]))
        mask.append(interesting(self.newlist[index][1],img[0].shape))
        for i in range(self.batch_size):
            mask.append(mask[0])
            img.append(transforming(img[0]))
        return (torch.stack(img,dim=0),torch.stack(mask,dim=0))
print('experimental')
#train_loader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=2)
train_loader = ExperimentalDataset(img_dir=image_path,mask_dir=mask_path,batch=batch_size,train='train')
print('loaders')
#val_loader = DataLoader(val_dataset, batch_size, shuffle=False, num_workers=2)
val_loader = ExperimentalDataset(img_dir=image_path,mask_dir=mask_path,batch=batch_size,train='validation')
#Модель для сегментации (вскоре будет изменена)
model = torchvision.models.segmentation.deeplabv3_resnet50(weights=torchvision.models.segmentation.DeepLabV3_ResNet50_Weights.DEFAULT,
                                                    progress=True)
print('model')

               id data_series    chart_type
0  000b92c3b098_x     abc;def  vertical_bar
1  000b92c3b098_y     0.0;1.0  vertical_bar
2  007a18eb4e09_x     abc;def  vertical_bar
3  007a18eb4e09_y     0.0;1.0  vertical_bar
4  00dcf883a459_x     abc;def  vertical_bar
['bars', 'boxplots', 'dot points', 'lines', 'scatter points']
experimental
loaders


Downloading: "https://download.pytorch.org/models/deeplabv3_resnet50_coco-cd0a2569.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet50_coco-cd0a2569.pth
100%|██████████| 161M/161M [00:00<00:00, 183MB/s]


model


In [11]:
from torchvision.models.segmentation.deeplabv3 import DeepLabHead
from torchvision.models.segmentation.fcn import FCNHead
model.classifier = DeepLabHead(2048, 23)
model.aux_classifier = FCNHead(1024, 23)
model = model.to(device)

from torch.nn import CrossEntropyLoss
import torch.nn.functional as F
#Функция ошибки, оптимизатор
loss = CrossEntropyLoss()#.to(device)
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [12]:
def pixel_accuracy(mask, output):
    output_softmax = F.softmax(output, dim=1)
    output_argmax = torch.argmax(output_softmax, dim=1)
#softmax(zi) = exp(zi)/sum[k:1..K](exp(zk))
#Свойства softmaxя
    bool_tensor = (torch.flatten(mask)) == (torch.flatten(output_argmax))

    return torch.sum(bool_tensor) / torch.numel(bool_tensor)

In [13]:
from tqdm import tqdm
epoch_count = 30

train_losses = []
val_losses = []

train_accs = []
val_accs = []
es_steps = 3
count_steps = 0

train_len = len(train_loader)
val_len = len(val_loader)
print(train_len)
print(val_len)

best_score = 1e10
#Обучение и валидация (надеюсь, не нужно описывать всё, что здесь происходит)
for epoch in range(epoch_count):
    if count_steps >= es_steps:
        print("Early stopping!")
        break

    train_loss_sum = 0
    train_pixel_acc = 0

    model.train()
    for l,(img_batch, mask_batch) in tqdm(train_loader):
#        print(epoch)
        img_batch, mask_batch = img_transform(img_batch, mask_batch, is_val=False)
#        img_batch = img_batch.to(device, non_blocking=True)
#        mask_batch = mask_batch.to(device, non_blocking=True)

        optimizer.zero_grad()

        output_batch = model(img_batch)
        loss_value = loss(output_batch['out'], mask_batch)

        train_pixel_acc += pixel_accuracy(mask_batch, output_batch['out']).detach()
        train_loss_sum += loss_value.detach()

        loss_value.backward()
        optimizer.step()
        
        del output_batch

    train_loss = train_loss_sum / train_len
    train_acc = train_pixel_acc / train_len

    train_losses.append(train_loss)
    train_accs.append(train_acc)

    print(f"Epoch {epoch} / {epoch_count} | train loss = {train_loss} | train acc = {train_acc}")

    model.eval()

    val_loss_sum = 0
    val_pixel_acc = 0

    for img_batch, mask_batch in (val_loader):
        img_batch, mask_batch = img_transform(img_batch, mask_batch, is_val=True)
#        img_batch = img_batch.to(device, non_blocking=True)
#        mask_batch = mask_batch.to(device, non_blocking=True)

        output_batch = model(img_batch)
        loss_value = loss(output_batch['out'], mask_batch)

        val_loss_sum = val_loss_sum + loss_value.detach()
        val_pixel_acc = val_pixel_acc + pixel_accuracy(mask_batch, output_batch['out']).detach()
        
        del output_batch

    val_loss = val_loss_sum / val_len
    val_acc = val_pixel_acc / val_len

    val_losses.append(val_loss)
    val_accs.append(val_acc)
    print(f"Epoch {epoch} / {epoch_count} | val loss = {val_loss} | val acc = {val_acc}")

    if val_loss < best_score:
        best_score = val_loss
        count_steps = 0
        torch.save(model, "best_model1.pt")
    else:
        count_steps += 1

54520
6057


  0%|          | 0/54520 [00:00<?, ?it/s]

img:  torch.Size([3, 480, 640]) 



ValueError: too many values to unpack (expected 2)

In [ ]:
import matplotlib.pyplot as plt
train_losses = [x.cpu().item() for x in train_losses]
val_losses = [x.cpu().item() for x in val_losses]
plt.plot(train_losses, linestyle="-")
plt.plot(val_losses, linestyle="--")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

In [ ]:
train_accs = [x.cpu().item() for x in train_accs]
val_accs = [x.cpu().item() for x in val_accs]

plt.plot(train_accs, linestyle="-")
plt.plot(val_accs, linestyle="--")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.show()

In [ ]:
model.eval()
label_map = np.array([
    (0, 0, 0),  # unlabeled
    (128, 64, 128),  # chart_title
    (130, 76, 0),  # axis_title
    (0, 102, 0),  # tick_label
    (112, 103, 87),  # plot-bb
    (28, 42, 168),  # axes
    (48, 41, 30),  # bars
    (0, 50, 89), # boxplots
    (107, 142, 35),  # dot points
    (70, 70, 70),  # lines
    (102, 102, 156),  # scatter points
    (255, 0, 0), # conflicting
])

def draw_segmentation_map(outputs):
    labels = torch.argmax(outputs.squeeze(), dim=0).numpy()
  
    # Create 3 Numpy arrays containing zeros.
    # Later each pixel will be filled with respective red, green, and blue pixels
    # depending on the predicted class.
  
    red_map   = np.zeros_like(labels).astype(np.uint8)
    green_map = np.zeros_like(labels).astype(np.uint8)
    blue_map  = np.zeros_like(labels).astype(np.uint8)
  
    for label_num in range(0, len(label_map)):
        index = labels == label_num
         
        R, G, B = label_map[label_num]
  
        red_map[index]   = R
        green_map[index] = G
        blue_map[index]  = B
  
    segmentation_map = np.stack([red_map, green_map, blue_map], axis=2)
    return segmentation_map

def image_overlay(image, segmented_image):
    alpha = 1  # transparency for the original image
    beta  = 0.8  # transparency for the segmentation map
    gamma = 0  # scalar added to each sum
  
    image = np.array(image)
    segmented_image = cv2.cvtColor(segmented_image, cv2.COLOR_RGB2BGR)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
     
    cv2.addWeighted(image, alpha, segmented_image, beta, gamma, image)
  
    return image

imgs_paths = os.listdir(image_path)
imgs_paths.sort()

def perform_inference(model=model ,imgs_paths=imgs_paths, num_images=10, image_dir='/kaggle/input/benetech-making-graphs-accessible/train/images/', device='cpu'):
     
#    out_table = pd.DataFrame(np.array([np.array(['nothing','nothing','nothing'])]),columns=['id','data_series','chart_type'])
     
    device = device if device is not None else ("cuda" if torch.cuda.is_available() else "cpu")
    
    tdb = []

    model.to(device)
    #preprocess = transforms.Compose([
    #  transforms.Resize([520, 520]),
    #  transforms.ToTensor(),
    #  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    #])
    # Load image handles for the validation set.
  
    # Randomly select 'num_images' from the whole set for inference.
    selected_images = imgs_paths
  
    # Iterate over selected images
    for img_name in selected_images:
         
        # Load and pre-process image.
        xsdb = [img_name[-4:]+'_x',"",""]
        ysdb = [img_name[-4:]+'_y',"",""]
        image_path = os.path.join(image_dir, img_name)
        img_raw = Image.open(image_path).convert("RGB")
        W, H = img_raw.size[:2]
        img_t = img_transform(img_raw)
        img_t = torch.unsqueeze(img_t, dim=0).to(device)
  
        # Model Inference
        with torch.no_grad():
            output = model(img_t)["out"].cpu()
  
        # Get RGB segmentation map
        segmented_image = draw_segmentation_map(output)
        ct, (at1, at2), (y_tick_fragments, x_tick_fragments), pb, axes, boxplot_fragments typeOf = script(img_t,segmented_image)
        
        xsdb[2]=typeOf
        ysdb[2]=typeOf
        
        xsdb[1]=';'.join([mmocr.models.textrecog.recognize(k) for k in x_tick_fragments])
        ysdb[1]=';'.join([mmocr.models.textrecog.recognize(k) for k in y_tick_fragments])
        
        # Resize to original image size
        segmented_image = cv2.resize(segmented_image, (W, H), cv2.INTER_LINEAR)
        overlayed_image = image_overlay(img_raw, segmented_image)
        
        tdb.append(np.array(xsdb))
        tdb.append(np.array(ysdb))
         
        # Plot
        plt.figure(figsize=(12, 10), dpi=100)
        plt.subplot(1, 3, 1)
        plt.axis("off")
        plt.title("Image")
        plt.imshow(np.asarray(img_raw))
  
        plt.subplot(1, 3, 2)
        plt.title("Segmentation")
        plt.axis("off")
        plt.imshow(segmented_image)
  
        plt.subplot(1, 3, 3)
        plt.title("Overlayed")
        plt.axis("off")
        plt.imshow(overlayed_image[:, :, ::-1])
         
        plt.show()
        plt.close()

  
    return np.array(tdb)

table_db=perform_inference()
table_db=pd.DataFrame(table_db)
table_db.to_csv('/kaggle/output/sample_submission.csv')

In [ ]:
'''import torch
import torchvision as tv
import os
import cv2
import numpy as np

def vertical_steps(arr: np.ndarray, flag_arr: np.ndarray, x, y):
    y_end = 0
    steps_spent = 0

    while arr[x][y + steps_spent] == [0, 255, 13]:
        flag_arr[x][y + steps_spent] = True
        steps_spent += 1

    y_end = y + steps_spent
    
    return [x, y_end], steps_spent, flag_arr

def horizontal_steps(arr: np.ndarray, flag_arr: np.ndarray, x, y):
    x_end = 0
    steps_spent = 0

    while arr[x + steps_spent][y] == [0, 255, 13]:
        flag_arr[x + steps_spent][y] = True
        steps_spent += 1

    x_end = x + steps_spent

    return [x_end, y], steps_spent, flag_arr

def script(tensor: np.ndarray):
    tensor_shape = np.shape(tensor)
    tensor_shape = (tensor_shape[0], tensor_shape[1], 1)
    flag_arr = np.zeros(tensor_shape)
    final_tensors = []

    for i in range(len(tensor)):
        for j in range(len(tensor[i])):
            if tensor[i][j] == [0, 255, 13] and not flag_arr[i][j]:
                a = (i, j)
                b, b_steps, flag_arr = horizontal_steps(tensor, flag_arr, i, j)
                c, c_steps, flag_arr = vertical_steps(tensor, flag_arr, i, j)

                if b_steps < c_steps:
                    d, _, flag_arr = vertical_steps(tensor, b[0], b[1])
                else:
                    d, _, flag_arr = horizontal_steps(tensor, c[0], c[1])

                final_tensor = tensor[b[1]:c[1], a[0]:d[0]]
                final_tensors.append(final_tensor)

    return final_tensors

arr = np.random.randint(0, 255, size=(90,90,3))
script(arr)'''